# 🦜🔗 解读LangChain核心源码：关于LLM和Agent（下）

# 课程开始

## 涉及的资源

### 🌹 智能体相关类整理

**（1）智能体类型**
- AgentType
    - ZERO_SHOT_REACT_DESCRIPTION（`ReAct`的一般实现）
    - REACT_DOCSTORE（ReAct，支持RAG）
    - SELF_ASK_WITH_SEARCH（使用`search 工具`不断反思获得答案）
    - CONVERSATIONAL_REACT_DESCRIPTION（`ReAct`，支持对话）
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION（同上）
    - CHAT_CONVERSATIONAL_REACT_DESCRIPTION（同上）
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION（`ReAct`，为对话模型优化，支持多输入）
    - OPENAI_FUNCTIONS（支持`OpenAI Function Calling`）
    - OPENAI_MULTI_FUNCTIONS（支持`OpenAI Function Calling`，支持多函数调度）

----
**（2）智能体执行器子组件：**
- `AgentAction`
    - `AgentActionMessageLog`
- `AgentStep`
- `AgentFinish`

----
**（3）单动智能体：**
- `BaseSingleActionAgent`
    - RunnableAgent
    - LLMSingleActionAgent（__deprecated__：`create_***_agent`）
    - XMLAgent（__deprecated__：create_xml_agent）
    - Agent
        - ChatAgent（__deprecated__：`create_react_agent`）
        - ConversationalAgent（__deprecated__：`create_react_agent`）
        - ConversationalChatAgent（__deprecated__：`create_json_chat_agent`）
        - StructuredChatAgent（__deprecated__：`create_structured_chat_agent`）
        - ZeroShotAgent（__deprecated__：`create_react_agent`）
        - ReActDocstoreAgent（__deprecated__）
            - ReActTextWorldAgent（__deprecated__）
        - SelfAskWithSearchAgent（__deprecated__：`create_self_ask_with_search`）
    - OpenAIFunctionsAgent（__deprecated__：`create_openai_functions_agent`）

----
**（4）多动智能体：**
- BaseMultiActionAgent
    - RunnableMultiActionAgent
    - OpenAIMultiFunctionsAgent（__deprecated__：`create_openai_tools_agent`）

----
**（5）Assistant：**
- Runnable
    - RunnableSerializable
        - Chain
            - `AgentExecutor`
                - MRKLChain（__deprecated__）
                - ReActChain（__deprecated__）
                - SelfAskWithSearchChain（__deprecated__）
        - `OpenAIAssistantRunnable`

----
**（6）提示语模板：**
- Runnable
    - RunnableSerializable
        - BasePromptTemplate [Dict, PromptValue]
            - BaseChatPromptTemplate
                - ChatPromptTemplate
                    - `AgentScratchPadChatPromptTemplate`

----
**（7）输出解析：**
- Runnable
    - RunnableSerializable
        - BaseOutputParser
            - `AgentOutputParser`
            - `MultiActionAgentOutputParser`

### 🌹 涉及的 LangChain 智能体

- 🌹 [LangChain 中的 Agent](https://python.langchain.com/docs/modules/agents/agent_types/)

|langchain智能体|支持多工具|支持多参数|推理过程优化|
|:---|:---:|:---:|:---:|
|create_openai_tools_agent|支持|支持|依赖大模型能力|
|create_react_agent|不支持|不支持|可以支持|
|自定义CoT|可以支持|可以支持|可以支持|

### 🌹 涉及的 LangChain 智能体相关源码

- 🌹 [查看 langchain/agents/openai_tools/base.py 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/openai_tools/base.py#L15-L97)
- 🌹 [查看 convert_to_openai_tool 的实现源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/utils/function_calling.py#L323-L341)
- 🌹 [langhain官方解析openai回调参数的方法](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/openai_tools.py)
- 🌹 [查看 create_react_agent 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/react/agent.py#L16)
- 🌹 [查看 ReAct 输出解析的源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/react_single_input.py#L22-L95)
- 🌹 [查看 AgentExcutor 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/agent.py#L1413-L1458)
- 🌹 [Chain](https://python.langchain.com/docs/modules/chains)

# （二）解读源码，拆解和自定义智能体

<div class="alert alert-info">
    <b>干货从这里开始！</b><br>
    接下来的例子中，会穿插 langchian 源码解读。
</div>

In [ ]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

## 4、解读 OpenAI 工具回调风格智能体

### ✍️ create_openai_executor：拆解 OpenAI 工具回调智能体的定义过程

![还是那个故事，但这次我们让老大爷聪明一点...如果可以的话。](./madongmei.gif)

In [ ]:
# from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub
from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool, convert_to_openai_function
import re

@tool
def ask_neighber(query: str) -> str:
    """想问你找的人住哪里就问我吧，我是楼下老大爷"""
    if(re.search("马冬梅", query)):
        return "马冬梅住在楼上322。"
    else:
        return "我不清楚"

def create_openai_executor(llm, tools):
    """
    使用openai智能体定义一个应用
    """
    # 定义 prompt
    prompt = hub.pull("hwchase17/openai-tools-agent")
    # 定义 Agent
    agent = create_openai_tools_agent(llm, tools, prompt)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=False)

    return executor

neighber = create_openai_executor(ChatZhipuAI(), [ask_neighber])

### 🌹 prompt： 观察 OpenAI 智能体的提示语

#### （1）从 Langsmith 的 hub 下载 hwchase17/openai-tools-agent

[🔗 查看 hub.pull("hwchase17/openai-tools-agent")](https://smith.langchain.com/hub/hwchase17/openai-tools-agent)

#### （2）等价的自定义 Prompt 模板

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

In [ ]:
prompt.invoke({
    "input": "请问马冬梅的家在哪里？",
    "chat_history": [],
    "agent_scratchpad": []
})

### 🌹 agent：阅读 create_openai_tools_agent 源码

[🔗 查看 langchain/agents/openai_tools/base.py 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/openai_tools/base.py#L15-L97)

```python
def create_openai_tools_agent(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
    """Create an agent that uses OpenAI tools."""

    missing_vars = {"agent_scratchpad"}.difference(prompt.input_variables)
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    llm_with_tools = llm.bind(tools=[convert_to_openai_tool(tool) for tool in tools])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            )
        )
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
    )
    return agent
```

### 🌹 executor： 阅读 AgentExcutor 源码

<div class="alert alert-warning">
    <b>注意：</b><br>
    LangChain中 的 Agent 与我们日常讨论的 Agent 在概念上是有区别的。
</div>

[🔗 查看 AgentExcutor 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/agent.py#L1413-L1458)

- 重点看 `_call` 方法实现
- 作为 Chain 子类，AgentExcutor 覆写了 stream 方法。
- 代码中的运行逻辑非常复杂，因此我们可以通过下面的方式研究其实际运行过程


### 🌹 run：观察 OpenAI 智能体的运行过程

#### （1）简单执行：invoke

In [ ]:
# invoke
neighber.invoke({"input":"马冬梅住哪里"})

#### （2）流输出：stream（仅智能体中的流）

In [ ]:
# stream
for s in neighber.stream({"input":"马冬梅住哪里"}):
    print(s)

#### （3）还是要使用事件流：astream_events

**看看方法 astream_events 的能力：**

In [ ]:
# astream_events
async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    print(e['name'], e['tags'], e['event'])

#### （4）观察与大模型的交互过程

只读取 **on_tool_end** 和 **on_chat_model_end** 两个事件：

In [ ]:
from langchain_zhipu import ChatZhipuAI
neighber = create_neighber(ChatZhipuAI())

async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("INPUT:")
            print(e['data']['input'])
        if("output" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("OUTPUT:")
            print(e['data']['output'])
        # print("\n", e)

#### （5）总结智能体的定义过程

- STEP-1 请求智能体（langchain -> ZhipuAI）: 发送带有Tools的请求
- STEP-2 智能体解析（ZhipuAI -> langchain）: 收到Tools-Calling消息
- STEP-3 调用本地工具（langchain）: 调用工具
- STEP-4 重新请求智能体（langchain） -> ZhipuAI）: 提交调用结果
- STEP-5 智能体最终生成（ZhipuAI -> langchain）: 大模型重新生成结果

### ✍️ 一步一步执行：就像我们常对大模型说的那样

#### （1）STEP-1 请求智能体（langchain -> ZhipuAI）: 发送带有Tools的请求

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

input = "马冬梅住在哪里？"
history = []

llm = ChatZhipuAI(tools=[convert_to_openai_tool(ask_neighber)])

agent = prompt | llm

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    <b>convert_to_openai_function</b> 和 <b>convert_to_openai_tool</b> 的区别是什么？
</div>

**🌞 参考：**
- [🔗 查看 convert_to_openai_tool 的实现源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/utils/function_calling.py#L323-L341)


In [ ]:
convert_to_openai_function(ask_neighber)

In [ ]:
convert_to_openai_tool(ask_neighber)

#### （2）STEP-2 智能体解析（ZhipuAI -> langchain）: 收到Tools-Calling消息

In [ ]:
steps_info = []
resp_llm = agent.invoke({"input": input, "chat_history": history, "agent_scratchpad": steps_info})
resp_llm

#### （3）STEP-3 调用本地工具（langchain）: 调用工具

In [ ]:
resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"]

In [ ]:
import json
tool_args = json.loads(resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"])
tool_args

In [ ]:
resp_tool = ask_neighber.invoke(tool_args)
resp_tool

[🔗 查看 langhain官方解析openai回调参数的方法](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/openai_tools.py)

#### （4）STEP-4 重新请求智能体（langchain） -> ZhipuAI）: 提交调用结果

In [ ]:
steps_info.append(input)
steps_info.append(resp_llm)
steps_info.append(resp_tool)
steps_info

#### （5）STEP-5 智能体最终生成（ZhipuAI -> langchain）: 大模型重新生成结果

In [ ]:
resp_llm = agent.invoke({"input": input, "chat_history": history, "agent_scratchpad": steps_info})
resp_llm

### 🌹 从源码中看 openai 作为先驱的影响

- [partners/openai/langchain_openai/chat_models/base.py（OpenAI模型）](https://github.com/langchain-ai/langchain/blob/master/libs/partners/openai/langchain_openai/chat_models/base.py)
- [community/langchain_community/adapters/openai.py（OpenAI风格API转换）](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/adapters/openai.py)
- [core/langchain_core/utils/function_calling.py（OpenAI风格回调工具函数）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/utils/function_calling.py)
- [core/langchain_core/output_parsers/openai_tools.py（OpenAI风格输出解析）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_tools.py)
- ...

## 5、解读 ReAct 风格智能体

### 🌹 ReAct 智能体的核心是其提示语引导

[🔗 查看 hub.pull("hwchase17/react")](https://smith.langchain.com/hub/hwchase17/react)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

In [ ]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)

### ✍️ create_react_executor：拆解 react 智能体的核心结构

#### （1）定义 ReAct 智能体

<div class="alert alert-info">
    <b>⚠️ 注意：</b><br>
    ReAct 智能体提示语的核心在于：<b>请使用如下的输出格式...</b>
</div>

In [ ]:
# from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain import hub
from langchain.tools import tool
import re

@tool
def ask_neighber(query: str) -> str:
    """我是马冬梅的邻居老大爷，关于她的事情你可以问我"""
    if(re.search("马冬梅", query)):
        return "楼上322"
    else:
        return "我不清楚"

prompt_react = """
请尽你最大努力回答用户的问题。

你必须注意以下原则：
1. 在需要使用本地文件时，请务必使用相关工具查询，而不要编造文件名。
2. 请使用中文。

你可以访问如下工具：
{tools}

请使用如下的输出格式：

Question: 你必须回答的问题
Thought: 你为了完成任务必须采取的思考过程
Action: 又称为工具，必须是这些工具其中之一： {tool_names}
Action Input: 调用工具所使用的参数
Observation: 工具执行的结果
... (这些过程 Thought/Action/Action Input/Observation 可以重复执行N次)
Thought: 我现在知道答案了
Final Answer: 问题的最终答案

开始！

Question: {input}
Thought:{agent_scratchpad}
"""

def create_react_executor(llm, tools):
    # 定义 prompt
    prompt = PromptTemplate.from_template(prompt_react)
    # 定义 Agent
    agent = create_react_agent(llm, tools, prompt)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=False)
    
    return executor

neighber = create_react_executor(ChatZhipuAI(), [ask_neighber])

#### （2）简单执行：invoke

In [ ]:
# invoke
neighber.invoke({"input":"马冬梅住哪个房间"})

#### （3）大模型的流输出：stream VS astream_events

In [ ]:
for chunk in neighber.stream({"input": "马冬梅住哪个房间"}):
    print(chunk)

In [ ]:
async for event in neighber.astream_events({"input": "马冬梅住哪里"}, version="v1"):
    kind = event["event"]
    if kind in ["on_chat_model_stream"]:
        print(event["data"]["chunk"].content, end="_")
    else:
        print("\n", "-"*10, "event >> ", kind)

### 🌹 观察 ReAct 智能体的运行过程

只读取 **on_tool_end** 和 **on_chat_model_end** 两个事件：

In [ ]:
from langchain_zhipu import ChatZhipuAI
neighber = create_react_neighber(ChatZhipuAI())

async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    print_line = lambda : print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            input = e['data']['input']
            print_line()
            print("INPUT:")
            if(input is not None and "messages" in input):
                print(input["messages"][0][0].content)
            else:
                print(input)
        if("output" in e['data']):
            output = e['data']['output'] 
            print_line()
            print("OUTPUT:")
            if(output is not None and "generations" in output):
                print(output["generations"][0][0]["text"])
            else:
                print(output)
        # print("\n", e)

### 🌹 阅读源码，解读 ReAct 智能体

- `ReAct` 比 `OpenAI智能体` 多了必要的解析过程。

- [🔗 查看 create_react_agent 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/react/agent.py#L16)
- [🔗 查看 ReAct 输出解析的源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/react_single_input.py#L22-L95)

<div class="alert alert-warning">
    <b>思考：上面的提示语中为什么不修改这几个关键词？</b><br>
    <ul>
        <li>Final Answer</li>
        <li>Action</li>
        <li>Action Input</li>
        <li>Observation</li>
    </ul>
</div>

**重新审视 ReAct 提示语：**

In [ ]:
prompt_react = """
请尽你最大努力回答用户的问题。

你必须注意以下原则：
1. 在需要使用本地文件时，请务必使用相关工具查询，而不要编造文件名。
2. 请使用中文。

你可以访问如下工具：
{tools}

请使用如下的输出格式：

Question: 你必须回答的问题
Thought: 你为了完成任务必须采取的思考过程
Action: 又称为工具，必须是这些工具其中之一： {tool_names}
Action Input: 调用工具所使用的参数
Observation: 工具执行的结果
... (这些过程 Thought/Action/Action Input/Observation 可以重复执行N次)
Thought: 我现在知道答案了
Final Answer: 问题的最终答案

开始！

Question: {input}
Thought:{agent_scratchpad}
"""

## 6、在LCEL框架下重定义《手撕AutoGPT》中的智能体

### ✍️ 自定义基于思维链的提示语

<div class="alert alert-info">
    <b>⚠️ 注意：</b><br>
    CoT 智能体提示语的核心在于：<b>请使用如下的输出格式...</b>
    <br>
    这相当于提示语模板中的：step by step ...
</div>

In [ ]:
PROMPT_COT = """
你是强大的AI助手，可以使用工具与指令自动化解决问题。

你必须遵循以下约束来完成任务:
1. 每次你的决策只使用一种工具，你可以使用任意多次。
2. 确保你调用的指令或使用的工具在下述给定的工具列表中。
3. 确保你的回答不会包含违法或有侵犯性的信息。
4. 如果你已经完成所有任务，确保以"FINISH"指令结束。
5. 用中文思考和输出。
6. 如果执行某个指令或工具失败，尝试改变参数或参数格式再次调用。
7. 你生成的回复必须遵循上文中给定的事实信息。不可以编造信息。DO NOT MAKE UP INFORMATION.
8. 如果得到的结果不正确，尝试更换表达方式。
9. 已经得到的信息，不要反复查询。
10. 确保你生成的动作是可以精确执行的。动作做中可以包括具体方法和目标输出。
11. 看到一个概念时尝试获取它的准确定义，并分析从哪些输入可以得到它的具体取值。
12. 生成一个自然语言查询时，请在查询中包含全部的已知信息。
13. 在执行分析或计算动作前，确保该分析或计算中涉及的所有子概念都已经得到了定义。
14. 你不可以打印一个文件的全部内容，这样的操作代价太大，且会造成不可预期的后果，是被严格禁止的。
15. 不要向用户提问。
16. 在需要使用本地文件时，请务必使用相关工具查询，而不要编造文件名。

你的任务是:
{input}

你有非常优秀的逻辑分析能力，可以通过因果关系找到最优的解决方案。

你要参考之前的思考记录:
{agent_scratchpad}

你需要评估你的表现:
1. 尽你最大的努力，用你最好的水平，通过分析和检查，做出最好的决定。
2. 带着全局观，自我反思你计划与动作。
3. 考虑你之前的策略与决策来改善的你的计划。
4. 如果你反复得到相同的结果，修改你的计划和决策，避免死循环。
5. 如果你当前的动作无法获取到需要的信息，尝试展开关键概念的定义，再重新推理。

如果你必须选择工具才能完成任务，可以使用以下工具之一，它们又称为动作或actions:

{tools}

你必须根据以下格式说明，输出你的思考过程:
1. 关键概念: 任务中涉及的组合型概念或实体。已经明确获得取值的关键概念，将其取值完整备注在概念后。
2. 概念拆解: 将任务中的关键概念拆解为一系列待查询的子要素。每个关键概念一行，后接这个概念的子要素，每个子要素一行，行前以' -'开始。
3. 反思:
   - 自我反思，观察以前的执行记录，思考概念拆解是否完整、准确。
   - 一步步思考是否每一个的关键概念或要素的查询都得到了准确的结果。
   - 反思你已经得到哪个要素/概念。你得到的要素/概念取值是否正确。从当前的信息中还不能得到哪些要素/概念。
   - 每个反思一行，行前以' -'开始。
4. 思考: 观察执行记录和你的自我反思，并一步步思考
  （1）分析要素间的依赖关系，例如：
    i. 我是否需要先获得A的值/定义，才能通过A来获得B？
    ii. 如果我先获得A，是否可以通过A筛选B，减少穷举每个B的代价？
    iii. A和B是否存在在同一数据源中，我能否在获取A的同时获取B？
    iv. 是否还有更高效或更聪明的办法来查询一个概念或要素？
    v. 如果上一次尝试查询一个概念或要素时失败了，我是否可以尝试从另一个资源中再次查询？
    vi. 诸如此类，你可以扩展更多的思考 ...
  （2）根据以上分析，排列子要素间的查询优先级
  （3）找出当前需要获得取值的子要素
  注意，不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源！
5. 推理: 根据你的反思与思考，一步步推理被选择的子要素取值的获取方式。如果前一次的计划失败了，请检查输入中是否包含每个概念/要素的明确定义，并尝试细化你的查询描述。
6. 计划: 严格遵守以下规则，计划你的当前动作。
  （1）详细列出当前动作的执行计划。只计划一步的动作。PLAN ONE STEP ONLY!
  （2）一步步分析，包括数据源，对数据源的操作方式，对数据的分析方法。有哪些已知常量可以直接代入此次分析。
  （3）不要尝试计算文件的每一个元素，这种计算代价太高，是严格禁止的。你可以通过分析找到更有效的方法，比如条件筛选。
  （4）上述分析是否依赖某个要素的取值/定义，且该要素的取值/定义尚未获得。若果是，重新规划当前动作，确保所有依赖的要素的取值/定义都已经获得。
  （5）不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源。不要编造信息。DO NOT MAKE UP ANY INFORMATION!!!
  （6）确保你执行的动作涉及的所有要素都已获得确切的取值/定义。
  （7）如果全部子任务已完成，请用FINISH动作结束任务。
  （8）不要在这里使用JSON格式来描述计划的当前动作，这会与下面的输出重复，造成无法解析。

你必须根据以下格式说明，输出JSON格式，描述所选择执行的动作/工具/指令:
{action_format_instructions}
"""

<div class="alert alert-warning">
    <b>💡 思考：自定义的CoT智能体，与ReAct智能体在提示语方面的本质差别是什么？</b>
</div>

### ✍️ 自定义 OutputParser

In [ ]:
from typing import List, Optional, Dict, Any, Union, Callable
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents.agent import AgentOutputParser, AgentAction, AgentFinish
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.output_parsers import PydanticOutputParser

In [ ]:
class Action(BaseModel):
    name: str = Field(
        description="The name of tool or action: FINISH or Other tool names."
        )
    args: Optional[Dict[str, Any]] = Field(
        default=None,
        description="Parameters of tool or action are composed of names and values."
        )

# 解析Action
_action_output_parser = PydanticOutputParser(pydantic_object=Action)
_action_parser_format = _action_output_parser.get_format_instructions()

class ReasonOutputParser(AgentOutputParser):
    """解析单个动作的智能体action和输入参数。
    """

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        action: Action = _action_output_parser.invoke(text)
        name: Optional[str] = action.name
        args: Optional[Dict[str, Any]] = action.args if text is not None else "No Args"
        log: str = text if text is not None else ""

        if name == "FINISH":
            return AgentFinish(args, log)
        elif name is not None:
            return AgentAction(name, args, log)

    @property
    def _type(self) -> str:
        return "Chain-of-Thought"


<div class="alert alert-warning">
    <b>💡 思考：自定义的CoT智能体，与ReAct智能体在判断推理结束方面的策略差别是什么？</b>
</div>

### ✍️ FINISH

In [ ]:
@tool
def FINISH(output: str) -> str:
    """
    用于表示任务完成的占位符工具。
    Args:
        output - 这是你要输出的答案。
    """
    
    return output

### ✍️ 自定义一个智能体

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.prompts import PromptTemplate
from langchain.tools.render import render_text_description

def _prompt_creator(prompt: str) -> Callable[[List[str]], str]:
    def creator(tools: List[str]) -> str:
        # 请注意，智谱AI等国内大模型对于pydantic的参数解析并不友好，使用JSON描述参数时会误读
        # 因此，不要使用 render_text_description_and_args 来生成工具描述
        tools_format = render_text_description(tools)

        template = PromptTemplate.from_template(prompt)
        return template.partial(
            tools=tools_format,
            action_format_instructions=_action_parser_format,
        )

    return creator

# 基于 CoT 的智能体
def create_cot_agent(llm: Any, prompt: Optional[str] = None, tools: List[str] = []) -> Any:
    prompt_creator = _prompt_creator(PROMPT_COT)
    if prompt is not None:
        prompt_creator = _prompt_creator(prompt)

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"])
        )
        | prompt_creator(tools)
        | llm
        | ReasonOutputParser()
    )

    return agent

### ✍️ create_cot_executor：尝试运行

In [ ]:
def create_cot_executor(llm, tools):
    # 定义 Agent
    agent = create_cot_agent(llm, tools=tools)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False, handle_parsing_errors=False)
    
    return executor

neighber = create_cot_executor(ChatZhipuAI(), [ask_neighber, FINISH])

In [ ]:
async for chunk in neighber.astream_events({"input": "马冬梅家住哪个房间？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

# （三）再现《手撕AutoGPT》：langchain+智谱+智能体

In [ ]:
# 《手撕 AutoGPT》中的有趣小工具
from colorama import init, Fore, Back, Style
import sys

THOUGHT_COLOR = Fore.GREEN
OBSERVATION_COLOR = Fore.YELLOW
ROUND_COLOR = Fore.RED
CODE_COLOR = Fore.BLUE

def color_print(text, color=None, end="\n"):
    if color is not None:
        content = color + text + Style.RESET_ALL + end
    else:
        content = text + end
    sys.stdout.write(content)
    sys.stdout.flush()

## 7、定义工具

### ✍️ 列举文件

In [ ]:
work_dir = "./data"

In [ ]:
# from langchain_openai import ChatOpenAI
from langchain.tools import tool
import os
import re
import fnmatch

@tool
def list_files(args=None) -> str:
    """如果需要查询资料，就首先使用该工具探查本地文件夹的结构和内容，展示它的文件名和文件夹名"""

    # 工作目录
    print("当前目录为：", workd_dir)

    # 定义你想要过滤的模式
    patterns = ['*.pdf', "*.xlsx"]

    if(os.path.isdir(workd_dir)):
        all_files = os.listdir(work_dir)
        # 过滤出所有匹配的文件
        matching_files = [f for f in all_files for p in patterns if fnmatch.fnmatch(f, p)]

        return "\n".join(matching_files)
    else:
        return []

In [ ]:
print(list_files.invoke({}))

### ✍️ 查询文档

#### （1）准备

In [ ]:
from typing import List
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_zhipu import ChatZhipuAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import BaseMessage

In [ ]:
def get_file_extension(filename: str) -> str:
    return filename.split(".")[-1]

def format_docs(docs: List[str]) -> str:
    return "\n\n".join([d.page_content for d in docs])

def convert_message_to_str(message: Union[BaseMessage, str]) -> str:
    if isinstance(message, BaseMessage):
        return message.content
    else:
        return message

In [ ]:
class FileLoadFactory:
    @staticmethod
    def get_loader(filename: str):
        filename = filename.strip()
        ext = get_file_extension(filename)
        if ext == "pdf":
            return PyPDFLoader(filename)
        elif ext == "docx" or ext == "doc":
            return Docx2txtLoader(filename)
        else:
            raise NotImplementedError(f"File extension {ext} not supported.")

In [ ]:
def load_docs(filename: str) -> List[Document]:
    file_loader = FileLoadFactory.get_loader(filename)
    return file_loader.load_and_split()

#### （2）使用 RAG 查询文档

In [ ]:
@tool
def ask_document(
        filename: str,
        query: str,
) -> str:
    """
    查询Word或PDF文档中的文本内容，以便回答问题。
    考虑上下文信息，确保问题对相关概念的定义表述完整。
    """

    path = os.path.join(work_dir, filename)
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"

    chunks = load_docs(path)
    # print(chunks)
    if chunks is None or len(chunks) == 0:
        return "无法读取文档内容"

    db = Chroma.from_documents(chunks, OpenAIEmbeddings())

    DEFAULT_QA_CHAIN_PROMPT = """
        你要严格依据如下资料回答问题，你的回答不能与其冲突，更不要编造。
        请始终使用中文回答。
        
        {context}
        
        问题: {question}
        """
    prompt = ChatPromptTemplate.from_template(DEFAULT_QA_CHAIN_PROMPT)

    qa_chain = (
        {
            "context": (lambda x: convert_message_to_str(x)) | db.as_retriever() | format_docs,
            "question": lambda x: convert_message_to_str(x)
        }
        | prompt
        | ChatZhipuAI()
    )

    # response = qa_chain.invoke(query)
    final_output = ""
    for chunk in qa_chain.stream(query):
        print(chunk.content, end="|")
        final_output += chunk.content
        
    return final_output

<div class="alert alert-warning">
    <b>思考：BaseTool 的定义中没有流方法，所以工具定义中也没有流输出支持</b><br>
    这将导致智能体在调用时，实际上无法获得流式输出（尽管可以在后台调试时打印，但不方便传递）。<br>
    这是为什么呢？？
</div>

In [ ]:
ask_document.invoke({"filename": "供应商资格要求.pdf", "query": "供应商达标标准"})

### ✍️ Excel 结构探查

In [ ]:
import pandas as pd

In [ ]:
def get_sheet_names(
        filename : str
) -> str :
    """获取 Excel 文件的工作表名称"""
    path = filename
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"
        
    excel_file = pd.ExcelFile(path.strip())
    sheet_names = excel_file.sheet_names
    return f"这是 '{path}' 文件的工作表名称：\n\n{sheet_names}"

In [ ]:
def get_column_names(
        filename : str
) -> str:
    """获取 Excel 文件的列名"""

    # 读取 Excel 文件的第一个工作表
    path = filename
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"
        
    df = pd.read_excel(path.strip(), sheet_name=0)  # sheet_name=0 表示第一个工作表
    column_names = '\n'.join(
        df.columns.to_list()
    )

    result = f"这是 '{path.strip()}' 文件第一个工作表的列名：\n\n{column_names}"
    return result

In [ ]:
def get_first_n_rows(
        filename : str,
        n : int = 3
) -> str :
    path = os.path.join(work_dir, filename)
    path = path.strip()
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"

    result = get_sheet_names(path)+"\n\n"
    result += get_column_names(path)+"\n\n"

    # 读取 Excel 文件的第一个工作表
    df = pd.read_excel(path, sheet_name=0)  # sheet_name=0 表示第一个工作表
    n_lines = '\n'.join(
        df.head(n).to_string(index=False, header=True).split('\n')
    )

    result += f"这是 '{path}' 文件第一个工作表的前{n}行样例：\n\n{n_lines}"
    return result

@tool
def inspect_excel(
        filename : str,
        n : int = 3
) -> str :
    """
    探查Excel数据文件的内容和结构，展示它的列名和前n行，n默认为3。
    注意，该工具仅使用于探查Excel文件，不能探查PDF或Word文件。
    
    使用该函数时应当准备提供filename和n两个参数，其中：
    
    - filename：要探查的Excel文件名
    - n: 默认的行数
    
    """
    return get_first_n_rows(filename, n)

In [ ]:
##
print(inspect_excel({"filename": "2023年8月-9月销售记录.xlsx"}))

### ✍️ Excel 数据分析

#### （1）准备

In [ ]:
import re
from langchain.tools import StructuredTool
from langchain_core.output_parsers import BaseOutputParser

# from Utils.PythonExecUtil import execute_python_code
from langchain_openai import ChatOpenAI
from langchain_experimental.utilities import PythonREPL

#### （2）自定义一个OutputParse

In [ ]:
class PythonCodeParser(BaseOutputParser):
    """从大模型返回的文本中提取Python代码。"""

    def _remove_marked_lines(self, input_str: str) -> str:
        lines = input_str.strip().split('\n')
        if lines and lines[0].strip().startswith('```'):
            del lines[0]
        if lines and lines[-1].strip().startswith('```'):
            del lines[-1]

        ans = '\n'.join(lines)
        return ans

    def parse(self, text: str) -> str:
        # 使用正则表达式找到所有的Python代码块
        python_code_blocks = re.findall(r'```python\n(.*?)\n```', text, re.DOTALL)
        # 从re返回结果提取出Python代码文本
        python_code = None
        if len(python_code_blocks) > 0:
            python_code = python_code_blocks[0]
            python_code = self._remove_marked_lines(python_code)
        return python_code

#### （3）定义提示语模板

In [ ]:
from langchain.prompts import PromptTemplate

excel_analyser_prompt = PromptTemplate.from_template("""
你的任务是先分析，再生成代码。

请根据用户的输入，一步步分析：
（1）用户的输入是否依赖某个条件，而这个条件没有明确赋值？
（2）我是否需要对某个变量的值做假设？
（3）已经从用户的输入中拆解概念，将其中包含的数字或实体名称，映射为所生成的函数入参，并在代码中使用？
（4）不能生成用户输入中没有包含的函数入参，将导致严重后果，这一点是否已经确认？

如果我需要对某个变量的值做假设，请直接输出：
```python
print("我需要知道____的值，才能生成代码。请完善你的查询。") # 请将____替换为需要假设的的条件
```
否则，创建Python代码，分析指定文件的内容。

MUST 请不要使用filename作为入参变量，直接写死在代码里即可。

MUST 你生成代码中所有的常量都必须来自我给你的信息或来自文件本身。不要编造任何常量。
如果常量缺失，你的代码将无法运行。你可以拒绝生成代码，但是不要生成编造的代码。
确保你生成的代码最终以print的方式输出结果(回答用户的问题)。

MUST 你可以使用的库只包括：pandas, re, math, datetime, openpyxl
确保你的代码只使用上述库，否则你的代码将无法运行。

MUST 确保你的代码可以通过运行的。

给定文件为：
{filename}

文件内容样例：
{inspections}

你输出的Python代码前后必须有markdown标识符，如下所示：
```python
# example code
```

用户输入：
{query}
""")

#### （4）定义执行链

In [ ]:
llm = ChatZhipuAI()
analysis_chain = excel_analyser_prompt | llm | PythonCodeParser()

#### （5）生成 python 代码并执行

In [ ]:
import ast
import types
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_experimental.utilities import PythonREPL

@tool
def excel_analyse(query: str, filename: str):
    """如果给定一个Excel文件，就可以根据该工具分析其内容。"""

    path = os.path.join(work_dir, filename)
    path = path.strip()
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"

    # columns = get_column_names(filename)
    inspections = get_first_n_rows(filename)

    # 打印详细信息
    color_print("\n#!/usr/bin/env python", CODE_COLOR, end="\n")

    # 生成代码
    code = ""
    for c in analysis_chain.stream({
        "query": query,
        "filename": path,
        "inspections": inspections
    }):
        ## 打印详细信息
        color_print(c, CODE_COLOR, end="")
        ## 收集代码成果
        code += c

    if code:        
        # 执行代码
        return PythonREPL().run(code)
    else:
        return "没有找到可执行的Python代码"

In [ ]:
##
excel_analyse.invoke({
    "query": "销售总额是多少?",
    "filename": "2023年8月-9月销售记录.xlsx"
})

## 8、对比这几个 🦜🔗LangChain 智能体

|langchain智能体|支持多工具|支持多参数|推理过程优化|
|:---|:---:|:---:|:---:|
|create_openai_tools_agent|支持|支持|依赖大模型能力|
|create_react_agent|不支持|不支持|可以支持|
|自定义CoT|可以支持|可以支持|可以支持|

### ✍️ GLM4 + create_openai_executor

openai风格的工具回调智能体：由大模型负责推理，决定何时以及如何使用工具。

In [ ]:
glm_openai = create_openai_executor(
    ChatZhipuAI(model="glm-4"),
    [
        # 列举本地文档
        list_files,
        # RAG 查询文档
        ask_document,
        # 探查 Excel 文件
        inspect_excel,
        # Excel 数据分析
        excel_analyse,
    ]
)

In [ ]:
async for chunk in glm_openai.astream_events({"input": "供应商达标的业绩要求是什么？"}, version="v1"):
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("INPUT:")
            print(e['data']['input'])
        if("output" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("OUTPUT:")
            print(e['data']['output'])

### ✍️ GLM4 + create_react_agent

ReAct智能体：由提示语影响大模型推理，通过规则制定、例子、小样本等**提示工程**影响推理过程。

<div class="alert alert-warning">
    <b>思考：</b><br>
    为什么 langchain 内置 的 create_react_agent 无法解析多个输入参数？
    <br>
    你有办法实现多参数输入吗？
</div>

**参考**

- [🔗 查看 create_react_agent: output_parser](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/react/agent.py#L125)
- [🔗 查看 ReActSingleInputOutputParser](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/react_single_input.py)

In [ ]:
zhipu_react = create_react_executor(
    ChatZhipuAI(model="glm-4"),
    [
        # 列举本地文档
        list_files,
        # RAG 查询文档
        ask_document,
        # 探查 Excel 文件
        inspect_excel,
        # Excel 数据分析
        excel_analyse,
    ]
)

In [ ]:
async for chunk in zhipu_react.astream_events({"input": "供应商达标的业绩要求是什么？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

### ✍️ GLM4 + create_cot_agent

自定义智能体：与 ReAct 类似，但通过思维链引导大模型的推理过程，并不仅仅停留在是否选择工具方面的思考，可通过提示语引导其对概念的深度拆解，就像是《手撕AutoGPT》中演示的那样。

In [ ]:
zhipu_cot = create_cot_executor(
    ChatZhipuAI(model="glm-4"),
    [
        # 列举本地文档
        list_files,
        # RAG 查询文档
        ask_document,
        # 探查 Excel 文件
        inspect_excel,
        # Excel 数据分析
        excel_analyse,
        # 结束工具
        FINISH,
    ]
)

In [ ]:
async for chunk in zhipu_cot.astream_events({"input": "供应商达标的标准是什么？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

In [ ]:
async for chunk in zhipu_cot.astream_events({"input": "9月份有哪些供货商达标？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

### ✍️ OpenAI / GPT4 + create_cot_agent

实践证明，GPT4 确实比 GLM4 具备更强的推理能力。<br>
但也不必气馁：
- 一是国内大模型每3-6个月会发布一次更新，逐步追赶GPT；
- 二是对业务领域做一定的提示语调优可以弥补推理能力。

In [ ]:
gpt_cot = create_cot_executor(
    ChatOpenAI(model="gpt-4-0125-preview"),
    [
        # 列举本地文档
        list_files,
        # RAG 查询文档
        ask_document,
        # 探查 Excel 文件
        inspect_excel,
        # Excel 数据分析
        excel_analyse,
        # 结束工具
        FINISH,
    ]
)

In [ ]:

async for chunk in gpt_cot.astream_events({"input": "供应商达标的标准是什么？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

In [ ]:

async for chunk in gpt_cot.astream_events({"input": "9月份有哪些供货商达标？？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

# ❤️ 知识点小结

- 解构源码 create_openai_tools_agent 并实践
- 解构源码 create_react_agent 并实践
- 学习如何自定义符合 AgentExecutor 的 AutoGPT 智能体

**以及：**

|langchain智能体|支持多工具|支持多参数|推理过程优化|
|:---|:---:|:---:|:---:|
|create_openai_tools_agent|支持|支持|依赖大模型能力|
|create_react_agent|不支持|不支持|可以支持|
|自定义CoT|可以支持|可以支持|可以支持|

# 结束了

## 🌈 彩蛋

### 1、推荐使用 LangGraph
- Runnable + Chain 时代：优点是方便管理状态；缺点是无逻辑定制空间；
- Runnable + LCEL 时代：优点是自定义执行逻辑；缺点是不方便管理状态（不是不能）；
- Runnable + LCEL + Langgraph 时代：全是优点；硬要说缺点，是要求你熟悉一点图的知识。

**langgraph 比 LCEL 增加的能力**
- 增加对智能体逻辑控制的能力
- 增加全局状态管理（智能体和工具都可以参与管理）
- 增加对记忆的深度管理
- 增加对工具的流式输出控制

**Langgraph 中一些较新的智能体论文实践...**

- [Agent Supervisor](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb)
- [Hierarchical Agent Teams](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/hierarchical_agent_teams.ipynb)
- [Multi Agent Collaboration](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/multi-agent-collaboration.ipynb)
- [Plan And Execute](https://github.com/langchain-ai/langgraph/blob/main/examples/plan-and-execute/plan-and-execute.ipynb)
- [ReWoo](https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb)
- [Reflexion](https://github.com/langchain-ai/langgraph/blob/main/examples/reflexion/reflexion.ipynb)
- [Self Discover](https://github.com/langchain-ai/langgraph/blob/main/examples/self-discover/self-discover.ipynb)
- ...

### 2、关于 RunnableLambda

<div class="alert alert-info">
    <b>💡 有趣的代码实验</b><br>
    问题1 你可以从这个示例扩展出更复杂的逻辑吗？<br>
    问题2 为何 RunnableLambda 可以实现流输出？<br>    
</div>

In [ ]:
from langchain_core.runnables import chain
import re

@chain
def choose_llm(input: str):
    if(re.search("GLM", input)):
        return ChatZhipuAI()
    else:
        return ChatOpenAI()

In [ ]:
for x in choose_llm.stream("请告诉我你来自哪里？GPT吗？"):
    print(x.content, end="|")

In [ ]:
for x in choose_llm.stream("请告诉我你来自哪里？GLM吗？"):
    print(x.content, end="|")